In [1]:
import os
import json
import math
import price
import ratios
import requests
import momentum
import pyticker
import dividend
import questrade
import auto_email
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import pandas_datareader.data as web

## Global Variables

In [2]:
WATCHLIST = ['O']
MOMENTUM_PERIODS = [1,3,6,12,24,36]

## Create S&P 500 Data If Not Exists

In [3]:
if not os.path.exists('./sp500_100_billions_symbols.json'):
    sp500 = pyticker.get_symbols_by_index('S&P 500')

    data = {'Symbol': [], 'Market_Cap (B)': []}

    market_cap_threshold = 200

    count = 0

    for symbol in sp500:
        count += 1
        print(f"{symbol}: {count}/{len(sp500)}")
        try:
            market_cap = ratios.calculate_market_cap(symbol)
        except:
            continue

        if market_cap >= market_cap_threshold and dividend.exists_dividends(symbol):
            data['Symbol'].append(symbol)
            data['Market_Cap (B)'].append(market_cap)
            print(f"{symbol} has been added!")

    with open('./sp500_100_billions_symbols.json', 'w') as fp:
        json.dump(data,fp)

## Load Data

In [4]:
with open('./sp500_100_billions_symbols.json', 'r') as fp:
    data = json.load(fp)
df = pd.DataFrame(data)
df.set_index('Symbol', inplace=True)

## Update Watchlist

In [5]:
watchlist_data = {'Symbol': [], 'Market_Cap (B)': []}
for symbol in WATCHLIST:
    if len(WATCHLIST) == 1:
        watchlist_data['Symbol'] = symbol
        watchlist_data['Market_Cap (B)'] = ratios.calculate_market_cap(symbol)
    else:
        watchlist_data['Symbol'].append(symbol)
        watchlist_data['Market_Cap (B)'].append(ratios.calculate_market_cap(symbol))

df.reset_index(inplace=True)
df = df.append(watchlist_data, ignore_index=True)
df.set_index('Symbol', inplace=True)


## Financials

In [6]:
for symbol in list(df.index):

    try:
        div_growth = dividend.calcualte_avg_dividend_growth(symbol,10)
        df.loc[symbol, 'Dividend_Growth'] = div_growth
    except:
        df.loc[symbol, 'Dividend_Growth'] = np.nan

    try:
        div_yield = dividend.calculate_current_dividend_yield(symbol)
        df.loc[symbol, 'Dividend_Yield'] = div_yield
    except:
        df.loc[symbol, 'Dividend_Yield'] = np.nan
        
    try:
        mom = momentum.calculate_equal_weight_momentum(symbol, MOMENTUM_PERIODS)
        df.loc[symbol,'Momentum'] = mom
    except:
        df.loc[symbol,'Momentum'] = np.nan

## Export DataFrame

In [7]:
df.to_csv(r'./export_df_rpi.csv')

## Email Alert

In [8]:
for symbol in list(df.index):
    high = price.calculate_prev_max_high(symbol,252)
    curr_price = price.get_current_price(symbol)
    df.loc[symbol,'12M_High'] = high
    df.loc[symbol,'Current_Price'] = curr_price
    df.loc[symbol,'15%_Drop'] = high * 0.85
    df.loc[symbol,'30%_Drop'] = high * 0.70
    df.loc[symbol,'50%_Drop'] = high * 0.5    
df

                  High         Low        Open       Close       Volume  \
Date                                                                      
1980-12-12    0.128906    0.128348    0.128348    0.128348  469033600.0   
1980-12-15    0.122210    0.121652    0.122210    0.121652  175884800.0   
1980-12-16    0.113281    0.112723    0.113281    0.112723  105728000.0   
1980-12-17    0.116071    0.115513    0.115513    0.115513   86441600.0   
1980-12-18    0.119420    0.118862    0.118862    0.118862   73449600.0   
...                ...         ...         ...         ...          ...   
2021-04-21  133.750000  131.300003  132.360001  133.500000   68847100.0   
2021-04-22  134.149994  131.410004  133.039993  131.940002   84566500.0   
2021-04-23  135.119995  132.160004  132.160004  134.320007   78657500.0   
2021-04-26  135.059998  133.559998  134.830002  134.720001   66684300.0   
2021-04-27  135.399994  134.110001  135.009995  134.389999   63819400.0   

             Adj_Close  

                  High         Low        Open       Close        Volume  \
Date                                                                       
1986-03-13    0.101563    0.088542    0.088542    0.097222  1.031789e+09   
1986-03-14    0.102431    0.097222    0.097222    0.100694  3.081600e+08   
1986-03-17    0.103299    0.100694    0.100694    0.102431  1.331712e+08   
1986-03-18    0.103299    0.098958    0.102431    0.099826  6.776640e+07   
1986-03-19    0.100694    0.097222    0.099826    0.098090  4.789440e+07   
...                ...         ...         ...         ...           ...   
2021-04-21  260.679993  257.250000  258.940002  260.579987  2.403040e+07   
2021-04-22  261.779999  255.639999  260.209991  257.170013  2.560620e+07   
2021-04-23  261.510010  257.269989  257.880005  261.149994  2.146260e+07   
2021-04-26  262.440002  260.170013  261.660004  261.549988  1.968210e+07   
2021-04-27  263.180115  260.119995  261.584991  261.970001  2.451259e+07   

           

                  High         Low        Open       Close     Volume  \
Date                                                                    
1972-08-25    0.064697    0.063477    0.063477    0.064453  2508800.0   
1972-08-28    0.064941    0.064209    0.064453    0.064209   972800.0   
1972-08-29    0.063965    0.063477    0.063965    0.063477  1945600.0   
1972-08-30    0.063477    0.062988    0.063477    0.063477   409600.0   
1972-08-31    0.062988    0.062500    0.062988    0.062500   870400.0   
...                ...         ...         ...         ...        ...   
2021-04-21  141.699997  140.529999  140.770004  141.199997  5778400.0   
2021-04-22  141.449997  139.419998  141.080002  139.669998  6568400.0   
2021-04-23  140.339996  139.029999  139.100006  139.899994  7657000.0   
2021-04-26  140.000000  137.639999  139.910004  137.910004  7576300.0   
2021-04-27  138.945007  137.550003  138.000000  138.380005  5084858.0   

             Adj_Close  
Date                    


                  High         Low        Open       Close       Volume  \
Date                                                                      
2006-05-25    4.605000    4.020000    4.030000    4.600000  395343000.0   
2006-05-26    4.674000    4.411000    4.630000    4.493000  103044000.0   
2006-05-30    4.498000    4.285000    4.497000    4.400000   49898000.0   
2006-05-31    4.536000    4.435000    4.435000    4.494000   30002000.0   
2006-06-01    4.810000    4.490000    4.493000    4.751000   62344000.0   
...                ...         ...         ...         ...          ...   
2021-04-21  384.100006  376.670013  378.589996  383.100006    2466300.0   
2021-04-22  389.500000  381.989990  383.079987  383.359985    3713700.0   
2021-04-23  389.880005  383.459991  384.350006  387.059998    2294800.0   
2021-04-26  392.940002  387.100006  387.829987  387.470001    2251100.0   
2021-04-27  390.429993  386.195007  387.829987  389.070007    2106519.0   

             Adj_Close  

,Market_Cap (B),Dividend_Growth,Dividend_Yield,Momentum,12M_High,Current_Price,15%_Drop,30%_Drop,50%_Drop
Symbol,,,,,,,,,
AAPL,2257.159507,0.238712,0.006009,0.761379,145.089996,134.389999,123.326497,101.562997,72.544998
CSCO,216.873121,0.328078,0.027837,0.225970,53.180000,51.369999,45.203000,37.226000,26.590000
KO,230.638500,0.064502,0.030608,0.151541,54.930000,53.580002,46.690500,38.451000,27.465000
XOM,238.474339,0.072788,0.061691,0.227362,62.549999,56.410000,53.167499,43.784999,31.275000
HD,346.899390,0.206233,0.018715,0.440559,328.829987,320.589996,279.505489,230.180991,164.414993
INTC,232.588800,0.078162,0.022770,0.230878,68.489998,57.970001,58.216498,47.942999,34.244999
JNJ,429.657717,0.065535,0.024390,0.187651,173.649994,163.179993,147.602495,121.554996,86.824997
JPM,457.758062,0.468526,0.023802,0.453991,161.690002,151.250000,137.436502,113.183002,80.845001
MSFT,1907.124687,0.144052,0.007978,0.574733,262.440002,261.970001,223.074002,183.708002,131.220001
